In [1]:
import pandas as pd
import numpy as np
import os

# 경고 메세지 off
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
# from matplotlib import font_manager, rc # 폰트 설정과 한글깨짐 방기
# font_path = "C:/Windows/Fonts/NGULIM.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)

from sklearn.metrics import silhouette_score # 성능 지표를 계산하기 위해 silhouette_score import
from sklearn.cluster import DBSCAN
import seaborn as sns
sns.set_style("darkgrid")
SEED=23

In [2]:
# Data load
address = './../data_processing/Dataset/'
data = pd.read_csv(os.path.join(address,'df_pm10_7day_result_1110.csv'), header = 0, encoding='cp949')

In [3]:
# in_serial = 장소 정보와 in_id =data sample id만 data_cd로 생성
data_cd = data[['in_serial','in_id']]
data_cd = data_cd.drop_duplicates()
data_cd = data_cd.set_index(keys='in_id')
# 클러스터링을 진행하기 위해 data table 형태 변환
df_pivot = pd.DataFrame()
for in_data_cd in data['in_id'].unique():
    sample = data[data['in_id'] == in_data_cd]
    time_index = []
    for i in range(0,len(sample)):
        time_index.append(i)
    sample['time_index'] = time_index
    df_pivot = pd.concat([df_pivot,sample], ignore_index=True)
df_pivot= df_pivot.pivot_table(index="time_index", columns="in_id", values="in_pm10_day")

#  1. 1일 데이터로 자르기

In [4]:
data_allSerial_1day = df_pivot.loc[:143]
data_allSerial_1day

in_id,ICL1L2000011,ICL1L2000017,ICL1L2000064,ICL1L2000065,ICL1L2000234,ICL1L2000235,ICL1L2000236,ICL1L2000237,ICL1L2000238,ICL1L2000239,...,ICW0W2100277,ICW0W2100278,IS70W2000822,IS70W2000851,IS70W2000852,IS70W2000853,IS70W2000854,IS70W2000855,IS70W2000856,IS70W2000858
time_index,,,,,,,,,,,,,,,,,,,,,
0,21.5,11.4,15.3,16.2,43.3,29.1,18.6,23.5,61.0,52.7,...,3.0,3.2,98.97,9.9,5.0,5.9,0.7,8.7,14.1,12.6
1,22.3,11.3,15.2,16.2,43.5,28.8,18.5,23.3,60.9,52.5,...,3.2,3.8,98.94,8.5,5.0,6.2,1.0,9.5,14.6,12.2
2,21.8,11.3,16.1,16.6,42.5,27.8,18.6,23.7,60.6,52.5,...,3.4,3.3,98.76,8.2,4.4,6.0,0.7,8.9,13.8,12.3
3,21.5,11.5,14.5,16.7,41.9,27.0,19.0,23.6,60.4,52.6,...,3.4,3.7,98.42,8.2,4.5,5.6,0.8,9.0,13.1,12.2
4,21.5,11.1,16.9,16.9,42.2,27.5,18.5,23.8,60.2,52.0,...,3.6,3.8,98.59,6.9,5.4,5.6,0.7,6.9,11.7,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,40.0,13.4,41.7,16.7,23.5,3.0,58.9,19.1,64.7,24.0,...,8.7,7.5,96.94,2.0,3.0,1.3,7.2,0.8,3.8,7.3
140,39.2,13.4,41.1,16.9,23.7,2.8,57.8,19.6,65.3,22.3,...,8.1,7.8,96.91,1.8,3.0,1.6,6.9,0.7,2.6,6.8
141,45.4,13.1,40.5,17.0,23.0,2.7,57.5,19.4,65.0,22.0,...,8.6,7.1,97.21,2.2,2.4,2.0,7.4,0.7,4.1,6.2


In [5]:
data_allSerial_1day.plot(legend = False, figsize=(16,6))
plt.tight_layout()

#  2. 각 컬럼 데이터 NaN 데이터 처리

In [6]:
# 선형보간법 사용
df_pivot_int = data_allSerial_1day.interpolate(method='linear') 
df_pivot_int_test = df_pivot_int.reindex(df_pivot_int.index, method="ffill")
df_pivot_int_test = df_pivot_int.reindex(df_pivot_int.index, method="bfill")
print("NaN processing (count):", np.isnan(data_allSerial_1day.values).sum(), "->", np.isnan(df_pivot_int_test.values).sum())

NaN processing (count): 45 -> 0


In [7]:
# Pivot Table
series_df = df_pivot_int_test.T
series_df = series_df.dropna(axis='columns')
series_df.head()

time_index,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
in_id,,,,,,,,,,,,,,,,,,,,,
ICL1L2000011,21.5,22.3,21.8,21.5,21.5,21.1,20.3,21.777778,22.4,21.9,...,39.4,41.2,33.8,41.4,41.3,40.0,39.2,45.4,44.3,43.1
ICL1L2000017,11.4,11.3,11.3,11.5,11.1,11.8,12.0,12.200000,12.4,13.1,...,11.7,12.5,12.6,12.8,12.9,13.4,13.4,13.1,13.3,14.8
ICL1L2000064,15.3,15.2,16.1,14.5,16.9,15.3,15.5,15.500000,15.9,16.8,...,37.8,38.0,40.3,40.6,41.5,41.7,41.1,40.5,41.3,42.2
ICL1L2000065,16.2,16.2,16.6,16.7,16.9,16.6,16.8,17.100000,17.3,17.3,...,17.8,17.1,17.7,16.2,16.4,16.7,16.9,17.0,16.3,15.8
ICL1L2000234,43.3,43.5,42.5,41.9,42.2,41.8,42.8,42.800000,42.9,43.0,...,24.3,24.1,23.6,23.7,23.9,23.5,23.7,23.0,22.0,22.0


In [8]:
# Scaling
from sklearn.preprocessing import StandardScaler # 데이터 정규화에 사용할 StandardScaler import
scaler = StandardScaler() # StandardScaler 정규화 객체 생성

scaled_ts = scaler.fit_transform(series_df.T).T # df_pivot에 대해 정규화를 진행함
scaled_ts_df = pd.DataFrame(scaled_ts)
namesofMySeries = list(series_df.index)

In [9]:
import sys
sys.path.append("../")
sys.path.append("../../")

from Clust.clust.ML.clustering.kMeans import KMeans

max_clusters = 9
kmeans_method_list =['euclidean', 'dtw', 'softdtw']
kmeans_method = kmeans_method_list[1]
data =scaled_ts_df.values
cluster_labels, metrics = KMeans().get_multipleCluster_result(data, max_clusters, kmeans_method)
metrics[["silhouette_score"]].plot()
metrics[["distortion_score"]].plot()

  0%|          | 0/7 [00:00<?, ?it/s]

Init 1


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 152 out of 152 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 304 out of 304 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 304 out of 304 | elapsed:    0.1s finished


54.663 --> 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 304 out of 304 | elapsed:    0.1s finished


...

9.919 --> 

[Parallel(n_jobs=1)]: Done 1216 out of 1216 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


9.919 --> 


[Parallel(n_jobs=1)]: Done 1216 out of 1216 | elapsed:    0.3s finished


[0, 144] [-2.5082852696402482, 9.483090506973959]


In [37]:
n_clusters = 3
model, cluster_labels = KMeans(). get_oneCluster_result(data, n_clusters, kmeans_method)

Init 1


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 152 out of 152 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 456 out of 456 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 456 out of 456 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 456 out of 456 | elapsed:    0.1s finished


38.458 --> 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 456 out of 456 | elapsed:    0.1s finished


18.159 --> 

...

15.983 --> 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 456 out of 456 | elapsed:    0.1s finished


15.983 --> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 456 out of 456 | elapsed:    0.1s finished


[0, 144] [-2.5082852696402482, 9.483090506973959]


In [33]:
file= 'ks_trained.pkl'

# Save model
model.to_pickle(file)

In [42]:
file= 'ks_trained.pkl'
import pickle
saved_model = pickle.dumps(file)

In [46]:
model = pickle.loads(saved_model)

In [34]:
# Load model
loded_model = KMeans().get_km_model(n_clusters, kmeans_method)
loded_model = loded_model.from_pickle(file)
model = loded_model

In [44]:
# Use loaded model to make predictions
cluster_labels = model.predict(data)

AttributeError: 'str' object has no attribute 'predict'

In [36]:
from Clust.clust.transformation.featureReduction.pca import show_clustering_result_by2DPCA
data_2d = show_clustering_result_by2DPCA(data, cluster_labels)

Dimension:  144 ----> 2


In [15]:
from Clust.clust.ML.clustering.util import count_lbel_info
count = count_label_info(cluster_labels)

ImportError: cannot import name 'count_lbel_info' from 'Clust.clust.ML.clustering.util' (../../Clust/clust/ML/clustering/util.py)